<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [27]:
import numpy as np
import pandas as pd

In [28]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [29]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

#### Defino mis funciones utiles

In [30]:
def getDocuments(corpus: list, getAsListofLists = True):
    '''Recibe una lista de documentos y retorna una lista de listas de documentos'''
    if getAsListofLists:
        return [[document] for document in corpus]
    return [document for document in corpus]

def getTerms(document: str , eliminateDuplicates = False):
    """recibe un documento tipo list y retorna una lista de palabras"""
    l = document.split(' ')
    if eliminateDuplicates:
        l = list(set(l))
    return l

def getCorpusWords(corpus: list):
    result = ''
    for v in corpus:
        result+= v+' '
    return getTerms(result.strip(),True)


In [31]:
# Las testeo:
documents = getDocuments(list(corpus))
terms = getTerms(documents[0][0])
corpusWords = getCorpusWords(list(corpus))

print('documents',documents)
print('terms',terms)
print('corpusWords',corpusWords)

documents [['que dia es hoy'], ['martes el dia de hoy es martes'], ['martes muchas gracias']]
terms ['que', 'dia', 'es', 'hoy']
corpusWords ['es', 'dia', 'muchas', 'el', 'de', 'hoy', 'que', 'martes', 'gracias']


### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [32]:
# Cada documento transformarlo en una lista de términos

doc2termsList = []
for document in getDocuments(list(corpus)):
    doc2termsList.append(getTerms(document[0]))
print(doc2termsList)


[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]


In [33]:
# Armar un vector de términos no repetidos de todos los documentos
corpusWords = getCorpusWords(list(corpus))
print(corpusWords)


['es', 'dia', 'muchas', 'el', 'de', 'hoy', 'que', 'martes', 'gracias']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [34]:
# Suponiendo lista de textos senteces y palabras words
documents = getDocuments(list(corpus),False)
ds = pd.DataFrame({"Textos": documents})
ds['Textos'] = ds['Textos']
ds.head()

,Textos
0,que dia es hoy
1,martes el dia de hoy es martes
2,martes muchas gracias


In [35]:
def oneHot(terms,documents):
    matriz = np.zeros((len(documents), len(terms)))
    for i, t in enumerate(documents):
        for j, p in enumerate(terms):
            if p in t:
                matriz[i, j] = 1
    matriz = matriz.astype(int)
    return pd.DataFrame(matriz, columns=terms)


terms = getCorpusWords(list(corpus))
documents = getDocuments(list(corpus),False)

df_concat = pd.concat([ds, oneHot(terms,documents)], axis=1, join='outer')

#tabla final
df_concat.head()

,Textos,es,dia,muchas,el,de,hoy,que,martes,gracias
0,que dia es hoy,1,1,0,0,0,1,1,0,0
1,martes el dia de hoy es martes,1,1,0,1,1,1,0,1,0
2,martes muchas gracias,1,0,1,0,0,0,0,1,1


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [36]:
def getFrec(texts: list, getwords):
    words = getwords(texts)
    df = pd.DataFrame(columns=words)
    wordsFre = []
    for i,text in enumerate(texts):
        for word in text.split(' '):
            df.loc[i] = [text.split(' ').count(word) for word in words]
    df_concat = pd.concat([pd.DataFrame(texts,columns=['Texts']),df ], axis=1, join='outer')
    return df_concat

texts = getDocuments(list(corpus),False)
getFrec(texts,getCorpusWords).head()


,Texts,es,dia,muchas,el,de,hoy,que,martes,gracias
0,que dia es hoy,1,1,0,0,0,1,1,0,0
1,martes el dia de hoy es martes,1,1,0,1,1,1,0,2,0
2,martes muchas gracias,0,0,1,0,0,0,0,1,1


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calcular_tfidf(texto): 
    tfidf_vectorizer = TfidfVectorizer() 
    matriz_tfidf = tfidf_vectorizer.fit_transform([texto]) 
    matriz_tfidf = matriz_tfidf.toarray()

    # Devolver la matriz de TF-IDF
    return matriz_tfidf

textos = getDocuments(list(corpus),False)

for texto in textos:
    matriz_tfidf = calcular_tfidf(texto)
    print('Texto:',texto, ' - Matriz',matriz_tfidf)



Texto: que dia es hoy  - Matriz [[0.5 0.5 0.5 0.5]]
Texto: martes el dia de hoy es martes  - Matriz [[0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.66666667]]
Texto: martes muchas gracias  - Matriz [[0.57735027 0.57735027 0.57735027]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def ordenar_por_similitud_cos(corpus, idx_doc):
    """
    Recibe un corpus de documentos y el índice de un documento, y devuelve los documentos
    ordenados por similitud coseno con respecto al documento correspondiente al índice dado.
    """
    # Creamos un vectorizador tf-idf para transformar el corpus en una matriz de características
    vectorizer = TfidfVectorizer()
    matriz_caracteristicas = vectorizer.fit_transform(corpus)

    # Calculamos la similitud coseno entre el documento dado y todos los demás documentos
    similitudes = cosine_similarity(matriz_caracteristicas[idx_doc], matriz_caracteristicas)

    # Obtenemos los índices de los documentos ordenados por similitud coseno descendente
    indices_ordenados = similitudes.argsort()[0][::-1]

    # Devolvemos los documentos ordenados por similitud coseno
    return [corpus[idx] for idx in indices_ordenados]


In [39]:
#Ejemplo

corpus = [
    'Este es el primer documento',
    'Este es el segundo documento',
    'Este es el tercer documento',
    'Este es el cuarto documento'
]

idx_doc = 3

documentos_ordenados = ordenar_por_similitud_cos(corpus, idx_doc)

print(documentos_ordenados)


['Este es el cuarto documento', 'Este es el tercer documento', 'Este es el segundo documento', 'Este es el primer documento']
